In [ ]:
import pandas as pd
import numpy as np
import luminol
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
file_encoding = 'cp1252'
df = pd.read_csv('data/lieky a pomocky.csv', error_bad_lines=False, encoding=file_encoding, delimiter=";")

In [ ]:
df.DOK_PRIJ_DAT = pd.to_datetime(df.DOK_PRIJ_DAT)
df.sort_values(by='DOK_PRIJ_DAT',inplace=True)
# dt = df.groupby(by="RGP_ID")['DOK_SK_UZN'].apply(list).to_frame()
dt = df

# Get unusual spikes in pharmancy counts

In [ ]:
ext = list(pd.read_csv('data/zoznam externych revizii.csv', error_bad_lines=False, encoding=file_encoding, delimiter=";").PZS_ID)

In [ ]:
dt = df
dt = dt.sort_values('DOK_PRIJ_DAT')
dt = dt[dt.DOK_PRIJ_DAT > '2016-12-31 07:30:00']
dt = dt[~dt.DOK_UKON_DAT.isna()]

# recode str to numbers
dt.DOK_MN = dt.DOK_MN.str.replace(",",".").astype(float)
dt.DOK_SK_UCT = dt.DOK_SK_UCT.str.replace(",",".").astype(float)
dt = dt[dt.DOK_MAX_CHYBA == 0]

# got rid of revised pharmacies
# not_checked_pzs = list(set(dt.PZS_ID.tolist()) - set(ext))
# dt = dt[dt.PZS_ID.isin(not_checked_pzs)]

In [ ]:
counter = 0
total_value = 0
absolute_value = 0
for lekaren in dt.DOK_ODD.unique():
    val = lekaren 
    # filter just current pzs
    testdf = dt.query("DOK_ODD == @val")
    testdf = testdf[['DOK_PRIJ_DAT','DOK_MN']]
    
    # transform
    ts = testdf.groupby('DOK_PRIJ_DAT').sum().reset_index()

    # get anomalies
    try:
        detector = luminol.anomaly_detector.AnomalyDetector(ts.DOK_MN.to_dict(), algorithm_name='bitmap_detector')
        anomalies = detector.get_anomalies()
    except:    
        continue
    
    #
    indices = []
    for i, item in enumerate(anomalies):
        if anomalies[i].anomaly_score > 11:
            indices.append(anomalies[i].start_timestamp)
    
    if len(indices) > 0:
        counter += 1
        absolute_value = ts.loc[indices].DOK_MN.sum() - ts.DOK_MN.mean()*len(indices)
        
        # compute economic value
        testdf_ = dt.query("DOK_ODD == @val")
        testdf_ = testdf_[['DOK_PRIJ_DAT','DOK_SK_UCT']]
    
        # transform
        ts_ = testdf_.groupby('DOK_PRIJ_DAT').sum().reset_index()
        economic_value = ts_.loc[indices].DOK_SK_UCT.sum() #- ts_.DOK_SK_UCT.mean()*len(indices)
        
        total_value += economic_value
        print("Lekaren {} has {} anomaly of {} drugs in total of {}".format(val, len(indices), \
                                                                            absolute_value.round(0), economic_value.round(0)))
            
        plt.figure(figsize=(10,5))
        plt.plot(ts.DOK_PRIJ_DAT,ts.DOK_MN)
        plt.plot(ts.loc[indices].DOK_PRIJ_DAT, ts.loc[indices].DOK_MN, 'r*')
        plt.show()
    
print("There is {} dok_odd with extreme anomalies with potential of {} saved in total.".format(counter, total_value.round(0)))

# Get unusual spikes in doctor vs # drugs

In [ ]:
counter = 0
total_value = 0
absolute_value = 0
for doktor in dt.DOK_KOD_LKR.unique():
    val = doktor 
    # filter just current pzs
    testdf = dt.query("DOK_KOD_LKR == @val")
    testdf = testdf[['DOK_PRIJ_DAT','DOK_MN']]
    
    # transform
    ts = testdf.groupby('DOK_PRIJ_DAT').sum().reset_index()

    # get anomalies
    try:
        detector = luminol.anomaly_detector.AnomalyDetector(ts.DOK_MN.to_dict(), algorithm_name='bitmap_detector')
        anomalies = detector.get_anomalies()
    except:    
        continue
    
    #
    indices = []
    for i, item in enumerate(anomalies):
        if anomalies[i].anomaly_score > 11:
            indices.append(anomalies[i].start_timestamp)
    
    if len(indices) > 0:
        counter += 1
        absolute_value = ts.loc[indices].DOK_MN.sum() - ts.DOK_MN.mean()*len(indices)
        
        # compute economic value
        testdf_ = dt.query("DOK_KOD_LKR == @val")
        testdf_ = testdf_[['DOK_PRIJ_DAT','DOK_SK_UCT']]
    
        # transform
        ts_ = testdf_.groupby('DOK_PRIJ_DAT').sum().reset_index()
        economic_value = ts_.loc[indices].DOK_SK_UCT.sum() #- ts_.DOK_SK_UCT.mean()*len(indices)
        
        total_value += economic_value
        print("Lekar {} has {} anomaly of {} drugs in total of {}".format(val, len(indices), \
                                                                            absolute_value.round(0), economic_value.round(0)))
            
        plt.figure(figsize=(10,5))
        plt.plot(ts.DOK_PRIJ_DAT,ts.DOK_MN)
        plt.plot(ts.loc[indices].DOK_PRIJ_DAT, ts.loc[indices].DOK_MN, 'r*')
        plt.show()
    
print("There is {} dok_odd with extreme anomalies with potential of {} saved in total.".format(counter, total_value.round(0)))